In [1]:
!pip install pytorch-optimizer
!pip install heavyball
!pip install fastfeedforward
!pip install triton
!pip install liger-kernel-nightly
!pip install jiwer
!pip install huggingface_hub safetensors

In [2]:
from huggingface_hub import login, HfApi, HfFolder
from safetensors.torch import save_file

# Log in to your Hugging Face account
login() # Follow the instructions to get your token

In [3]:
# For demonstration, we use synthetic dataset

import requests
import json

# URL containing the JSON data
url = 'https://huggingface.co/datasets/ChavyvAkvar/synthetic-number/resolve/main/corpus_texts_train.json?download=true'

# Send a GET request to the URL
response = requests.get(url)

texts = []

# Check if the request was successful
if response.status_code == 200:
    # Parse the JSON data
    texts = response.json()

    # Check if the data is a list
    if isinstance(texts, list):
        # Now `data` is a Python list (array)
        print(texts[5])
    else:
        print("The data is not an array.")
else:
    print(f"Failed to retrieve data: {response.status_code}")

print(len(texts))

seventy nine thousand one hundred and thirty nine seventy nine thousand one hundred and forty seventy nine thousand one hundred and forty one seventy nine thousand one hundred and forty two seventy nine thousand one hundred and forty three seventy nine thousand one hundred and forty four seventy nine thousand one hundred and forty five seventy nine thousand one hundred and forty six seventy nine thousand one hundred and forty seven seventy nine thousand one hundred and forty eight seventy nine thousand one hundred and forty nine seventy nine thousand one hundred and fifty seventy nine thousand one hundred and fifty one seventy nine thousand one hundred and fifty two seventy nine thousand one hundred and fifty three seventy nine thousand one hundred and fifty four seventy nine thousand one hundred and fifty five seventy nine thousand one hundred and fifty six seventy nine thousand one hundred and fifty seven seventy nine thousand one hundred and fifty eight seventy nine thousand one hun

In [4]:
# URL containing the JSON data
url = 'https://huggingface.co/datasets/ChavyvAkvar/synthetic-number/resolve/main/corpus_texts_val.json?download=true'

# Send a GET request to the URL
response = requests.get(url)

texts_val = []

# Check if the request was successful
if response.status_code == 200:
    # Parse the JSON data
    texts_val = response.json()

    # Check if the data is a list
    if isinstance(texts_val, list):
        # Now `data` is a Python list (array)
        print(texts_val[5])
    else:
        print("The data is not an array.")
else:
    print(f"Failed to retrieve data: {response.status_code}")

print(len(texts_val))

fifty eight thousand two hundred and eight fifty eight thousand two hundred and nine fifty eight thousand two hundred and ten fifty eight thousand two hundred and eleven fifty eight thousand two hundred and twelve fifty eight thousand two hundred and thirteen fifty eight thousand two hundred and fourteen fifty eight thousand two hundred and fifteen fifty eight thousand two hundred and sixteen fifty eight thousand two hundred and seventeen fifty eight thousand two hundred and eighteen fifty eight thousand two hundred and nineteen fifty eight thousand two hundred and twenty fifty eight thousand two hundred and twenty one fifty eight thousand two hundred and twenty two fifty eight thousand two hundred and twenty three fifty eight thousand two hundred and twenty four fifty eight thousand two hundred and twenty five fifty eight thousand two hundred and twenty six fifty eight thousand two hundred and twenty seven fifty eight thousand two hundred and twenty eight fifty eight thousand two hund

In [5]:
import torch
import torch.nn as nn
import math

def create_1d_rope_embeddings(seq_len, dim, device='cpu'):
    # Ensure the dimension is even
    assert dim % 2 == 0, "Dimension must be even for 1D RoPE"

    # Create position indices
    inv_freq = 1.0 / (10000 ** (torch.arange(0, dim, 2).float() / dim))
    inv_freq = inv_freq.to(device)

    # Create position embeddings
    sinusoid_inp = torch.einsum("i,j->ij", torch.arange(seq_len, dtype=torch.float, device=device), inv_freq)

    # Create the RoPE embeddings
    rope_sin = torch.sin(sinusoid_inp).unsqueeze(1).permute(1, 0, 2)
    rope_cos = torch.cos(sinusoid_inp).unsqueeze(1).permute(1, 0, 2)

    return rope_sin, rope_cos

# We don't use this implementation in practice because Liger Kernel implementation is better
def apply_1d_rope(token_embeddings, rope_sin, rope_cos):
    # Ensure the token embeddings dimension is even
    assert token_embeddings.shape[-1] % 2 == 0, "Token embeddings dimension must be even for 1D RoPE"

    # Split the token embeddings into even and odd dimensions
    token_embeddings_even = token_embeddings[..., 0::2]
    token_embeddings_odd = token_embeddings[..., 1::2]

    # Apply the RoPE embeddings
    token_embeddings_even_rotated = token_embeddings_even * rope_cos - token_embeddings_odd * rope_sin
    token_embeddings_odd_rotated = token_embeddings_even * rope_sin + token_embeddings_odd * rope_cos

    # Concatenate the rotated embeddings
    token_embeddings_rotated = torch.cat((token_embeddings_even_rotated, token_embeddings_odd_rotated), dim=-1)
    return token_embeddings_rotated

In [6]:
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers.pre_tokenizers import Whitespace

# Train a BPE tokenizer
tokenizer = Tokenizer(BPE(unk_token="[UNK]"))
tokenizer.pre_tokenizer = Whitespace()
trainer = BpeTrainer(vocab_size=500, special_tokens=["[UNK]", "[PAD]", "[EOS]"])
tokenizer.train_from_iterator(texts, trainer)

# Save and load the tokenizer
tokenizer.save("bpe_tokenizer.json")
tokenizer = Tokenizer.from_file("bpe_tokenizer.json")

In [7]:
# We don't use this implementation in practice because Liger Kernel implementation is better

import torch

class RMSNorm(torch.nn.Module):
    def __init__(self, dim, eps = 1e-6):
        super().__init__()
        self.weight = torch.nn.Parameter(torch.ones(dim))
        self.eps = eps

    def forward(self, x):
        dtype = x.dtype
        x = x.float()
        norm = torch.mean(x * x, dim=-1, keepdim=True)
        xnorm = x * torch.rsqrt(norm + self.eps)
        xnorm = xnorm.to(dtype=dtype)
        return xnorm * self.weight

def justnorm(self, x):
        #return F.normalize(x, p=2, dim=-1)
        res = x / x.norm(p=2, dim=-1, keepdim=True)
        return res

In [8]:
# We don't use this implementation in practice because Liger Kernel implementation is better

import torch
from torch import nn

class SwiGLU(nn.Module):
    def __init__(self, dimension):
        super().__init__()
        self.linear_1 = nn.Linear(dimension,dimension)
        self.linear_2 = nn.Linear(dimension,dimension)

    def forward(self, x):
        output = self.linear_1(x)
        swish = output * torch.sigmoid(output)
        swiglu = swish * self.linear_2(x)

        return swiglu

In [16]:
import torch.nn.functional as F
from fastfeedforward import FFF

# Liger Kernel 🐯
from liger_kernel.transformers import LigerRMSNorm, LigerLayerNorm, liger_rotary_pos_emb, LigerSwiGLUMLP

class GroupedAttentionLayer(nn.Module):
    def __init__(self, dim, num_heads, num_groups, max_seq_len, device='cpu'):
        super(GroupedAttentionLayer, self).__init__()
        self.dim = dim
        self.num_heads = num_heads
        self.num_groups = num_groups
        self.head_dim = dim // num_heads
        self.max_seq_len = max_seq_len
        self.device = device

        self.norm = LigerLayerNorm(dim // num_heads)

        # Linear projections for Q, K, V
        self.q_linear = nn.Linear(dim, dim, bias=True)
        self.k_linear = nn.Linear(dim, dim, bias=True)
        self.v_linear = nn.Linear(dim, dim, bias=True)
        self.out_linear = nn.Linear(dim, dim, bias=False)

        # nGPT stuff 😎
        self.sqk_init_value = 1.0
        self.sqk_init_scaling = 1.0 / dim ** 0.5
        self.sqk = torch.nn.Parameter(self.sqk_init_scaling*torch.ones(dim))

        self.sqrt_head_dim = (dim / num_heads) ** 0.5

    def forward(self, x):
        batch_size, seq_len, dim = x.shape

        # Precompute the RoPE embeddings
        rope_sin, rope_cos = create_1d_rope_embeddings(seq_len, self.head_dim, self.device)

        # Linear projections
        q = self.q_linear(x).view(batch_size, seq_len, self.num_heads, self.head_dim).transpose(1, 2)
        k = self.k_linear(x).view(batch_size, seq_len, self.num_heads, self.head_dim).transpose(1, 2)
        v = self.v_linear(x).view(batch_size, seq_len, self.num_heads, self.head_dim).transpose(1, 2)

        # Apply RoPE to Q, K
        q, k = liger_rotary_pos_emb(q, k, rope_cos, rope_sin)

        sqk = (self.sqk * (self.sqk_init_value/self.sqk_init_scaling)).view(1, self.num_heads, 1, self.dim // self.num_heads)
        q = sqk * self.norm(q)
        k = sqk * self.norm(k)

        softmax_scale = self.sqrt_head_dim

        # Group the queries, keys, and values
        q = q.reshape(batch_size, self.num_heads, self.num_groups, seq_len, self.head_dim // self.num_groups)
        k = k.reshape(batch_size, self.num_heads, self.num_groups, seq_len, self.head_dim // self.num_groups)
        v = v.reshape(batch_size, self.num_heads, self.num_groups, seq_len, self.head_dim // self.num_groups)

        # Compute attention using scaled_dot_product_attention
        attn_output = []
        for group in range(self.num_groups):
            attn_output.append(F.scaled_dot_product_attention(
                q[:, :, group], k[:, :, group], v[:, :, group], attn_mask=None, dropout_p=0.2, is_causal=True, scale=softmax_scale
            ))

        attn_output = torch.cat(attn_output, dim=2)

        # Reshape the output
        out = attn_output.transpose(1, 2).contiguous().view(batch_size, seq_len, dim)

        # Linear projection
        out = self.out_linear(out)

        return out

class TransformerLayer(nn.Module):
    def __init__(self, dim, num_heads, num_groups, max_seq_len, device='cpu'):
        super(TransformerLayer, self).__init__()
        self.attention = GroupedAttentionLayer(dim, num_heads, num_groups, max_seq_len, device)
        self.num_heads = num_heads
        self.dim = dim

        self.norm1 = LigerLayerNorm(dim)
        self.norm2 = LigerLayerNorm(dim)

        self.c_fc   = nn.Linear(dim, 2 * 4 *  dim)
        self.silu = nn.SiLU()

        # Fast-FFN 🔥
        self.depth = 4 # your choice of the FFF depth
        self.leaf_width = math.ceil(dim*4 / 2**self.depth)
        self.region_leak = 0.3 # your choice of the region leak probability (0 - 0.5) to encourage generalisation in very deep FFFs
        self.ffn = FFF(
            input_width=dim,
            leaf_width=self.leaf_width,
            output_width=dim,
            depth=self.depth,
            activation=torch.nn.SiLU(),
            dropout=0.2,
            region_leak=self.region_leak
        )

        # nGPT stuff 😎
        self.attn_alpha_init_value = 0.05
        self.attn_alpha_init_scaling = 1.0 / dim ** 0.5
        self.attn_alpha = torch.nn.Parameter(self.attn_alpha_init_scaling*torch.ones(dim))

        self.mlp_alpha_init_value = 0.05
        self.mlp_alpha_init_scaling = 1.0 / dim ** 0.5
        self.mlp_alpha = torch.nn.Parameter(self.mlp_alpha_init_scaling*torch.ones(dim))

        self.suv_init_value = 1.0
        self.suv_init_scaling = 1.0
        self.suv = torch.nn.Parameter(self.suv_init_scaling*torch.ones(2 * 4 * dim))

    def forward(self, x):
        lr = self.attn_alpha * (self.attn_alpha_init_value / self.attn_alpha_init_scaling)
        lr = torch.abs(lr)

        A_norm = self.norm1(x) # normally, normalization is not needed
        B_norm = self.norm1(self.attention(x))

        res = A_norm + lr * (B_norm - A_norm)
        x = self.norm1(res)

        xin = x

        uv = self.c_fc(xin)
        suv = (self.suv * ((self.suv_init_value/self.suv_init_scaling) * (self.num_heads ** 0.5)))

        uv = suv * uv

        u, v = torch.chunk(uv, 2, dim=-1)

        x_mlp = u * self.silu(v)

        x_mlp = self.ffn(x)

        lr = self.mlp_alpha * (self.mlp_alpha_init_value / self.mlp_alpha_init_scaling)
        lr = torch.abs(lr)

        A_norm = self.norm2(x) # normally, normalization is not needed
        B_norm = self.norm2(x_mlp)

        res = A_norm + lr * (B_norm - A_norm)
        x = self.norm2(res)
        return x

class TransformerModel(nn.Module):
    def __init__(self, num_layers, dim, num_heads, num_groups, max_seq_len, vocab_size, loss_fn, device='cpu'):
        super(TransformerModel, self).__init__()
        self.token_embedding = torch.nn.Embedding(vocab_size, dim)
        self.layers = nn.ModuleList([
            TransformerLayer(dim, num_heads, num_groups, max_seq_len, device)
            for _ in range(num_layers)
        ])
        self.head = nn.Linear(dim, vocab_size, bias=False)
        self.vocab_size = vocab_size
        self.loss_fn = loss_fn
        self.dim = dim

        self.apply(self._init_weights)
        print("number of parameters: %.2fM" % (self.get_num_params()/1e6,))

        # nGPT stuff 😎
        self.sz_init_value = 1.00
        self.sz_init_scaling = 1.0 / dim ** 0.5
        self.sz = torch.nn.Parameter(self.sz_init_scaling*torch.ones(vocab_size))

    def get_num_params(self, non_embedding=False):
        n_params = sum(p.numel() for p in self.parameters())
        # if non_embedding:
        #    n_params -= self.transformer.wpe.weight.numel()
        return n_params

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=1.0 / self.dim ** 0.5)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=1.0 / self.dim ** 0.5)

    def forward(self, x, targets=None):
        # Token embeddings
        x = self.token_embedding(x)

        # Apply transformer layers
        for layer in self.layers:
            x = layer(x)

        if targets is not None:
            logits = self.head(x)

            sz = self.sz * (self.sz_init_value/self.sz_init_scaling)
            logits = sz * logits
            loss = self.loss_fn(logits.view(-1, self.vocab_size), targets.view(-1))
        else:
            # inference-time mini-optimization: only forward the lm_head on the very last position
            logits = self.head(x[:, [-1], :]) # note: using list [-1] to preserve the time dim
            sz = self.sz * (self.sz_init_value/self.sz_init_scaling)
            logits = sz * logits
            loss = None

        return logits, loss

In [10]:
import torch

# Tokenize and pad the training texts
encoded = tokenizer.encode_batch(texts)
sequences = [torch.tensor(enc.ids, dtype=torch.long) for enc in encoded]
pad_token_id = tokenizer.token_to_id("[PAD]")
seq_len = max(len(seq) for seq in sequences)
sequences_padded = [torch.cat([seq, torch.tensor([pad_token_id] * (seq_len - len(seq)), dtype=torch.long)]) for seq in sequences]
inputs = torch.stack(sequences_padded)
targets = torch.stack([torch.cat([seq[1:], torch.tensor([pad_token_id], dtype=torch.long)]) for seq in sequences_padded])
train_dataset = torch.utils.data.TensorDataset(inputs, targets)
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True)

# Tokenize and pad the validation texts
encoded_val = tokenizer.encode_batch(texts_val)
sequences_val = [torch.tensor(enc.ids, dtype=torch.long) for enc in encoded_val]
seq_len_val = max(len(seq) for seq in sequences_val)
sequences_padded_val = [torch.cat([seq, torch.tensor([pad_token_id] * (seq_len_val - len(seq)), dtype=torch.long)]) for seq in sequences_val]
inputs_val = torch.stack(sequences_padded_val)
targets_val = torch.stack([torch.cat([seq[1:], torch.tensor([pad_token_id], dtype=torch.long)]) for seq in sequences_padded_val])
val_dataset = torch.utils.data.TensorDataset(inputs_val, targets_val)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=128, shuffle=False)

In [11]:
print(tokenizer.decode(inputs[5].squeeze().tolist(), skip_special_tokens=False))
print(tokenizer.decode(targets[5].squeeze().tolist(), skip_special_tokens=False))

seventy nine thousand one hundred and thirty nine seventy nine thousand one hundred and forty seventy nine thousand one hundred and forty one seventy nine thousand one hundred and forty two seventy nine thousand one hundred and forty three seventy nine thousand one hundred and forty four seventy nine thousand one hundred and forty five seventy nine thousand one hundred and forty six seventy nine thousand one hundred and forty seven seventy nine thousand one hundred and forty eight seventy nine thousand one hundred and forty nine seventy nine thousand one hundred and fifty seventy nine thousand one hundred and fifty one seventy nine thousand one hundred and fifty two seventy nine thousand one hundred and fifty three seventy nine thousand one hundred and fifty four seventy nine thousand one hundred and fifty five seventy nine thousand one hundred and fifty six seventy nine thousand one hundred and fifty seven seventy nine thousand one hundred and fifty eight seventy nine thousand one hun

In [12]:
from pytorch_optimizer import get_supported_optimizers

supported_optimizers = get_supported_optimizers()

print(supported_optimizers)

['a2grad', 'accsgd', 'adabelief', 'adabound', 'adadelta', 'adafactor', 'adahessian', 'adai', 'adalite', 'adalomo', 'adamax', 'adamg', 'adammini', 'adamod', 'adamp', 'adams', 'adamw', 'adan', 'adanorm', 'adapnm', 'adashift', 'adasmooth', 'ademamix', 'aggmo', 'aida', 'alig', 'amos', 'apollo', 'asgd', 'avagrad', 'bsam', 'came', 'dadaptadagrad', 'dadaptadam', 'dadaptadan', 'dadaptlion', 'dadaptsgd', 'diffgrad', 'fadam', 'fromage', 'galore', 'gravity', 'grokfastadamw', 'kate', 'lamb', 'lars', 'lion', 'lomo', 'madgrad', 'msvag', 'nero', 'novograd', 'padam', 'pid', 'pnm', 'prodigy', 'qhadam', 'qhm', 'radam', 'ranger', 'ranger21', 'scalableshampoo', 'schedulefreeadamw', 'schedulefreesgd', 'sgdp', 'sgdw', 'shampoo', 'signsgd', 'sm3', 'soap', 'sophiah', 'srmm', 'stableadamw', 'swats', 'tiger', 'yogi']


In [17]:
from pytorch_optimizer import create_optimizer
from liger_kernel.transformers import LigerCrossEntropyLoss
import heavyball

# cross entropy layers from liger kernel
loss_fn = LigerCrossEntropyLoss(ignore_index=tokenizer.token_to_id("[PAD]"))

# Training Loop
model = TransformerModel(num_layers=4, dim=128, num_heads=8, num_groups=4, max_seq_len=seq_len, vocab_size=tokenizer.get_vocab_size(), loss_fn=loss_fn, device='cuda')
model.to('cuda')

optimizer = heavyball.PrecondSchedulePaLMSOAP(model.parameters(), lr=1e-3)

# optimizer = create_optimizer(
#     model,
#     'soap',
#     lr=3e-3,
#     weight_decay=0.01,
#     betas=(.95, .95),
#     precondition_frequency=10
# )

# Training loop with validation
for epoch in range(10):
    model.train()  # Set the model to training mode
    total_loss = 0
    for batch in train_dataloader:
        input_batch, target_batch = batch
        input_batch = input_batch.to('cuda', dtype=torch.long)
        target_batch = target_batch.to('cuda', dtype=torch.long)
        logits, loss = model(input_batch, target_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    avg_train_loss = total_loss / len(train_dataloader)

    # Validation step
    model.eval()  # Set the model to evaluation mode
    total_val_loss = 0
    with torch.no_grad():  # Disable gradient calculation for validation
        for batch in val_dataloader:
            input_batch, target_batch = batch
            input_batch = input_batch.to('cuda', dtype=torch.long)
            target_batch = target_batch.to('cuda', dtype=torch.long)
            logits, loss = model(input_batch, target_batch)
            total_val_loss += loss.item()
    avg_val_loss = total_val_loss / len(val_dataloader)

    print(f"Epoch {epoch+1}, Train Loss: {avg_train_loss}, Val Loss: {avg_val_loss}")

number of parameters: 1.36M
Epoch 1, Train Loss: 2.6898239413394203, Val Loss: 1.8081824481487274
Epoch 2, Train Loss: 1.3279937143567242, Val Loss: 1.1273137927055359
Epoch 3, Train Loss: 0.736464325008513, Val Loss: 0.5984855145215988
Epoch 4, Train Loss: 0.3040381756764424, Val Loss: 0.29964106529951096
Epoch 5, Train Loss: 0.17356291809414007, Val Loss: 0.2193262279033661
Epoch 6, Train Loss: 0.13872316206180596, Val Loss: 0.17861077934503555
Epoch 7, Train Loss: 0.12209571803672405, Val Loss: 0.16574157029390335
Epoch 8, Train Loss: 0.11217661315127264, Val Loss: 0.1496550552546978
Epoch 9, Train Loss: 0.1051488764866998, Val Loss: 0.13852191343903542
Epoch 10, Train Loss: 0.09951280359225938, Val Loss: 0.13066082820296288


In [18]:
# Update learning rate for the second loop
for param_group in optimizer.param_groups:
    param_group['lr'] = 1e-4  # Reduce learning rate

# Second Training Loop with smaller learning rate
for epoch in range(5):  # Example for 5 additional epochs
    model.train()
    total_loss = 0
    for batch in train_dataloader:
        input_batch, target_batch = batch
        input_batch = input_batch.to('cuda', dtype=torch.long)
        target_batch = target_batch.to('cuda', dtype=torch.long)
        logits, loss = model(input_batch, target_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    avg_train_loss = total_loss / len(train_dataloader)

    # Validation step
    model.eval()
    total_val_loss = 0
    with torch.no_grad():
        for batch in val_dataloader:
            input_batch, target_batch = batch
            input_batch = input_batch.to('cuda', dtype=torch.long)
            target_batch = target_batch.to('cuda', dtype=torch.long)
            logits, loss = model(input_batch, target_batch)
            total_val_loss += loss.item()
    avg_val_loss = total_val_loss / len(val_dataloader)

    print(f"Epoch {epoch+11}, Train Loss: {avg_train_loss}, Val Loss: {avg_val_loss}")

Epoch 11, Train Loss: 0.09351658085479012, Val Loss: 0.12882744893431664
Epoch 12, Train Loss: 0.09196082087634486, Val Loss: 0.1278102770447731
Epoch 13, Train Loss: 0.09089936608377891, Val Loss: 0.1285855621099472
Epoch 14, Train Loss: 0.08973412574092045, Val Loss: 0.12705962359905243
Epoch 15, Train Loss: 0.08899183627925342, Val Loss: 0.12560377269983292


In [19]:
# URL containing the JSON data
url = 'https://huggingface.co/datasets/ChavyvAkvar/synthetic-number/resolve/main/corpus_texts_test.json?download=true'

# Send a GET request to the URL
response = requests.get(url)

texts_test = []

# Check if the request was successful
if response.status_code == 200:
    # Parse the JSON data
    texts_test = response.json()

    # Check if the data is a list
    if isinstance(texts_test, list):
        # Now `data` is a Python list (array)
        print(texts_test[5])
    else:
        print("The data is not an array.")
else:
    print(f"Failed to retrieve data: {response.status_code}")

print(len(texts_test))

eighty two thousand six hundred and fifty eighty two thousand six hundred and fifty one eighty two thousand six hundred and fifty two
50


In [20]:
# URL containing the JSON data
url = 'https://huggingface.co/datasets/ChavyvAkvar/synthetic-number/resolve/main/corpus_texts_answer.json?download=true'

# Send a GET request to the URL
response = requests.get(url)

texts_answer = []

# Check if the request was successful
if response.status_code == 200:
    # Parse the JSON data
    texts_answer = response.json()

    # Check if the data is a list
    if isinstance(texts_answer, list):
        # Now `data` is a Python list (array)
        print(texts_answer[5])
    else:
        print("The data is not an array.")
else:
    print(f"Failed to retrieve data: {response.status_code}")

print(len(texts_answer))

eighty two thousand six hundred and fifty eighty two thousand six hundred and fifty one eighty two thousand six hundred and fifty two eighty two thousand six hundred and fifty three eighty two thousand six hundred and fifty four eighty two thousand six hundred and fifty five eighty two thousand six hundred and fifty six eighty two thousand six hundred and fifty seven eighty two thousand six hundred and fifty eight eighty two thousand six hundred and fifty nine eighty two thousand six hundred and sixty eighty two thousand six hundred and sixty one eighty two thousand six hundred and sixty two eighty two thousand six hundred and sixty three eighty two thousand six hundred and sixty four eighty two thousand six hundred and sixty five eighty two thousand six hundred and sixty six eighty two thousand six hundred and sixty seven eighty two thousand six hundred and sixty eight eighty two thousand six hundred and sixty nine eighty two thousand six hundred and seventy eighty two thousand six hu

In [27]:
import torch
import jiwer

def infer(model, tokenizer, texts_test, texts_answer, max_length=200, device='cuda'):
    generated_texts = []
    wer_scores = []
    reference_texts = []

    for input_text, reference_text in zip(texts_test, texts_answer):
        # Tokenize the input text
        encoded = tokenizer.encode(input_text)
        input_ids = torch.tensor(encoded.ids, dtype=torch.long).unsqueeze(0).to(device)

        # Generate text
        generated_ids = input_ids
        with torch.no_grad():
            for _ in range(max_length):
                # Pass the input through the model
                outputs, loss = model(generated_ids)
                # Get the last token logits
                next_token_logits = outputs[:, -1, :]

                # Optionally adjust logits with temperature
                temperature = 1.0
                next_token_logits = next_token_logits / temperature

                # Convert logits to probabilities using softmax
                next_token_probs = torch.nn.functional.softmax(next_token_logits, dim=-1)

                # Sample the next token from the probability distribution
                next_token_id = torch.multinomial(next_token_probs, num_samples=1)

                # Append the next token to the sequence
                generated_ids = torch.cat([generated_ids, next_token_id], dim=-1)

                # Stop if the EOS token is generated
                if next_token_id.item() == tokenizer.token_to_id("[EOS]"):
                    break

        # Decode the generated sequence
        generated_ids = generated_ids.squeeze().tolist()
        decoded_text = tokenizer.decode(generated_ids, skip_special_tokens=True)
        generated_texts.append(decoded_text)

        reference_text = tokenizer.encode(reference_text)
        reference_text = reference_text.ids[:len(generated_ids)]
        reference_text = tokenizer.decode(reference_text, skip_special_tokens=True)

        reference_texts.append(reference_text)

        # Calculate WER
        wer = jiwer.wer(reference_text, decoded_text)
        wer_scores.append(wer)

    return generated_texts, reference_texts, wer_scores

generated_texts, reference_texts, wer_scores = infer(model, tokenizer, texts_test, texts_answer, max_length=200)

for i, (input_text, generated_text, reference_text, wer) in enumerate(zip(texts_test, generated_texts, reference_texts, wer_scores)):
    print(f"Input text {i+1}: {input_text}")
    print(f"Generated text {i+1}: {generated_text}")
    print(f"Reference text {i+1}: {reference_text}")
    print(f"WER {i+1}: {wer}")
    print()

Input text 1: seventy eight thousand three hundred and fifty one seventy eight thousand three hundred and fifty two seventy eight thousand three hundred and fifty three
Generated text 1: seventy eight thousand three hundred and fifty one seventy eight thousand three hundred and fifty two seventy eight thousand three hundred and fifty three eighty three thousand eight hundred and sixty two eighty three thousand eight hundred and sixty three eighty three thousand eight hundred and sixty four eighty three thousand eight hundred and sixty five eighty three thousand eight hundred and sixty six eighty three thousand eight hundred and sixty seven eighty three thousand eight hundred and sixty eight eighty three thousand eight hundred and sixty nine eighty three thousand eight hundred and seventy eighty three thousand eight hundred and seventy one eighty three thousand eight hundred and seventy two eighty three thousand eight hundred and seventy three eighty three thousand eight hundred and sev

In [28]:
# Save the model weights as safetensors
def save_safetensors(model, path):
    state_dict = model.state_dict()
    # Convert state_dict to safetensors format
    save_file(state_dict, path)

# Call the function to save weights in safetensors format
save_safetensors(model, "llm-numbers/pytorch_model.safetensors")  # Replace with your model name

# Push the model and tokenizer to Hugging Face
api = HfApi()
api.upload_folder(
    folder_path="llm-numbers",
    repo_id="ChavyvAkvar/llm-numbers",  # Replace with your username and model name
    repo_type="model",
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


pytorch_model.safetensors:   0%|          | 0.00/5.47M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/ChavyvAkvar/llm-numbers/commit/f938a4e69df02633b2676219070b65cba8f2c47e', commit_message='Upload folder using huggingface_hub', commit_description='', oid='f938a4e69df02633b2676219070b65cba8f2c47e', pr_url=None, repo_url=RepoUrl('https://huggingface.co/ChavyvAkvar/llm-numbers', endpoint='https://huggingface.co', repo_type='model', repo_id='ChavyvAkvar/llm-numbers'), pr_revision=None, pr_num=None)